In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the data
df = pd.read_excel('../Week2/SnD/snd.xlsx', sheet_name='Stats')
df.head()

,Date,Map,Offense,Defense,FBTeam,FBPlayer,FBTime,FBWeapon,FBTraded?,PlantSite,PlantClock,Winner,WinType,EndClock,Clutch?,Timeout,DefenseWinner?
0,2025-08-06,Firing Range,Wolves,OUG,OUG,Solo,01:44:00,LMG,No,NaN,NaN,OUG,Elim,00:18:00,NaN,NaN,True
1,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,01:40:00,Oden,Yes,A,01:10:00,Wolves,Elim,00:06:00,NaN,NaN,False
2,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,01:48:00,Oden,No,NaN,NaN,Wolves,Elim,01:26:00,NaN,NaN,False
3,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,00:31:00,Oden,Yes,NaN,NaN,OUG,Time,00:00:00,NaN,NaN,True
4,2025-08-06,Firing Range,Wolves,OUG,Wolves,Sound,01:50:00,Nade,No,A,01:28:00,Wolves,Elim,00:16:00,NaN,NaN,False


In [3]:
# Keep only date part of the 'Date' column
df['Date'] = df['Date'].dt.date

In [4]:
# Create MatchID column
df['MatchID'] = (
    df[['Date', 'Map', 'Offense', 'Defense']]
    .apply(lambda r: f"{r['Date']}_{r['Map']}_{'_'.join(sorted([r['Offense'], r['Defense']]))}", axis=1)
)
df

,Date,Map,Offense,Defense,FBTeam,FBPlayer,FBTime,FBWeapon,FBTraded?,PlantSite,PlantClock,Winner,WinType,EndClock,Clutch?,Timeout,DefenseWinner?,MatchID
0,2025-08-06,Firing Range,Wolves,OUG,OUG,Solo,01:44:00,LMG,No,NaN,NaN,OUG,Elim,00:18:00,NaN,NaN,True,2025-08-06_Firing Range_OUG_Wolves
1,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,01:40:00,Oden,Yes,A,01:10:00,Wolves,Elim,00:06:00,NaN,NaN,False,2025-08-06_Firing Range_OUG_Wolves
2,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,01:48:00,Oden,No,NaN,NaN,Wolves,Elim,01:26:00,NaN,NaN,False,2025-08-06_Firing Range_OUG_Wolves
3,2025-08-06,Firing Range,Wolves,OUG,Wolves,Pegg,00:31:00,Oden,Yes,NaN,NaN,OUG,Time,00:00:00,NaN,NaN,True,2025-08-06_Firing Range_OUG_Wolves
4,2025-08-06,Firing Range,Wolves,OUG,Wolves,Sound,01:50:00,Nade,No,A,01:28:00,Wolves,Elim,00:16:00,NaN,NaN,False,2025-08-06_Firing Range_OUG_Wolves
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2025-08-17,Coastal,Wolves,Q9,Wolves,Rise,01:53:00,Sniper,No,A,01:18:00,Wolves,Elim,00:13:00,NaN,Q9,False,2025-08-17_Coastal_Q9_Wolves
713,2025-08-17,Coastal,Wolves,Q9,Q9,Dchen,01:05:00,Sniper,No,A,00:18:00,Q9,Defuse,00:07:00,NaN,NaN,True,2025-08-17_Coastal_Q9_Wolves
714,2025-08-17,Coastal,Wolves,Q9,Q9,Dchen,00:59:00,Sniper,Yes,A,00:43:00,Wolves,Elim,00:08:00,NaN,NaN,False,2025-08-17_Coastal_Q9_Wolves
715,2025-08-17,Coastal,Wolves,Q9,Q9,Maoqi,01:34:00,Sniper,No,NaN,NaN,Q9,Elim,00:33:00,NaN,NaN,True,2025-08-17_Coastal_Q9_Wolves


In [5]:
# Collect halftime data
halftime_data = []

for match_id, group in df.groupby('MatchID'):
    group = group.reset_index(drop=True)
    teams = pd.unique(group[['Offense', 'Defense']].values.ravel())
    wins = {team: 0 for team in teams}

    for i, row in group.iterrows():
        wins[row['Winner']] += 1

        # After 8 rounds, record halftime stats
        if i == 7:
            halftime_data.append({
            'MatchID': match_id,
            'Map': row['Map'],
            'TeamA': teams[0],
            'TeamB': teams[1],
            'ScoreA': wins[teams[0]],
            'ScoreB': wins[teams[1]],
            'HalftimeSideA': 'Offense' if row['Offense'] == teams[0] else 'Defense',
            'HalftimeSideB': 'Offense' if row['Offense'] == teams[1] else 'Defense',
            'FinalWinner': group.iloc[-1]['Winner'],
            'Overtime': False
        })
            # If the match goes to overtime, we need to handle that
            if len(group) > 16:
                halftime_data[-1]['Overtime'] = True

            break

# Create halftime DataFrame
halftime_df = pd.DataFrame(halftime_data)

In [6]:
halftime_df

,MatchID,Map,TeamA,TeamB,ScoreA,ScoreB,HalftimeSideA,HalftimeSideB,FinalWinner,Overtime
0,2025-07-31_Kurohana_Q9_SPG,Kurohana,SPG,Q9,4,4,Offense,Defense,Q9,True
1,2025-08-01_Coastal_GodL_XROCK,Coastal,GodL,XROCK,3,5,Offense,Defense,XROCK,False
2,2025-08-01_Kurohana_SPG_Wolves,Kurohana,Wolves,SPG,2,6,Offense,Defense,SPG,False
3,2025-08-01_Standoff_GodL_XROCK,Standoff,XROCK,GodL,2,6,Offense,Defense,GodL,True
4,2025-08-01_Tunisia_SPG_Wolves,Tunisia,Wolves,SPG,3,5,Offense,Defense,SPG,True
5,2025-08-02_Coastal_GodL_OUG,Coastal,OUG,GodL,3,5,Offense,Defense,OUG,True
6,2025-08-02_Firing Range_Q9_Wolves,Firing Range,Q9,Wolves,3,5,Offense,Defense,Wolves,True
7,2025-08-02_Tunisia_Q9_Wolves,Tunisia,Wolves,Q9,2,6,Offense,Defense,Q9,False
8,2025-08-03_Coastal_OUG_Q9,Coastal,Q9,OUG,6,2,Offense,Defense,Q9,False
9,2025-08-03_Firing Range_SPG_XROCK,Firing Range,SPG,XROCK,7,1,Offense,Defense,SPG,False


In [21]:
# Calculate overall match win rates for each team
team_wins = halftime_df.groupby('FinalWinner').size()
team_total_matches = halftime_df.groupby('TeamA').size() + halftime_df.groupby('TeamB').size()
team_total_matches = team_total_matches.dropna()
team_win_rates = (team_wins / team_total_matches).fillna(0)



In [24]:
team_win_rates

AG        1.000000
DVS       0.500000
GodL      0.500000
OUG       0.384615
Q9        0.727273
SPG       0.571429
Soul      0.200000
Wolves    0.583333
XLR8      0.000000
XROCK     0.416667
Xceed     0.000000
dtype: float64

In [13]:
df = halftime_df.copy()

df['HalftimeDiff'] = df['ScoreA'] - df['ScoreB']            # from TeamA (first-attack) perspective

df['Target_TeamA_Win'] = (df['FinalWinner'] == df['TeamA']).astype(int)

# 4) Keep useful metadata for analysis/plots
model_tidy = df[['MatchID','Map','TeamA','TeamB','HalftimeDiff','FinalWinner','Overtime','Target_TeamA_Win']].copy()

model_tidy.head()


,MatchID,Map,TeamA,TeamB,HalftimeDiff,FinalWinner,Overtime,Target_TeamA_Win
0,2025-07-31_Kurohana_Q9_SPG,Kurohana,SPG,Q9,0,Q9,True,0
1,2025-08-01_Coastal_GodL_XROCK,Coastal,GodL,XROCK,-2,XROCK,False,0
2,2025-08-01_Kurohana_SPG_Wolves,Kurohana,Wolves,SPG,-4,SPG,False,0
3,2025-08-01_Standoff_GodL_XROCK,Standoff,XROCK,GodL,-4,GodL,True,0
4,2025-08-01_Tunisia_SPG_Wolves,Tunisia,Wolves,SPG,-2,SPG,True,0


In [8]:
comeback1 = (model_tidy['HalftimeDiff'] < 0) & (model_tidy['Target_TeamA_Win'] == 1)
comeback2 = (model_tidy['HalftimeDiff'] > 0) & (model_tidy['Target_TeamA_Win'] == 0)

model_tidy[comeback1 | comeback2]

,MatchID,Map,TeamA,TeamB,HalftimeDiff,FinalWinner,Overtime,Target_TeamA_Win
5,2025-08-02_Coastal_GodL_OUG,Coastal,OUG,GodL,-2,OUG,True,1
11,2025-08-03_Tunisia_OUG_Q9,Tunisia,Q9,OUG,-2,Q9,False,1
14,2025-08-06_Firing Range_OUG_Wolves,Firing Range,Wolves,OUG,2,OUG,True,0
19,2025-08-07_Firing Range_Wolves_XROCK,Firing Range,Wolves,XROCK,-2,Wolves,False,1
28,2025-08-14_Firing Range_AG_Soul,Firing Range,AG,Soul,-4,AG,True,1
39,2025-08-16_Tunisia_GodL_Soul,Tunisia,Soul,GodL,2,GodL,False,0
41,2025-08-17_Coastal_OUG_SPG,Coastal,OUG,SPG,2,SPG,False,0


In [9]:
len(model_tidy)

44

In [10]:
model_tidy

,MatchID,Map,TeamA,TeamB,HalftimeDiff,FinalWinner,Overtime,Target_TeamA_Win
0,2025-07-31_Kurohana_Q9_SPG,Kurohana,SPG,Q9,0,Q9,True,0
1,2025-08-01_Coastal_GodL_XROCK,Coastal,GodL,XROCK,-2,XROCK,False,0
2,2025-08-01_Kurohana_SPG_Wolves,Kurohana,Wolves,SPG,-4,SPG,False,0
3,2025-08-01_Standoff_GodL_XROCK,Standoff,XROCK,GodL,-4,GodL,True,0
4,2025-08-01_Tunisia_SPG_Wolves,Tunisia,Wolves,SPG,-2,SPG,True,0
5,2025-08-02_Coastal_GodL_OUG,Coastal,OUG,GodL,-2,OUG,True,1
6,2025-08-02_Firing Range_Q9_Wolves,Firing Range,Q9,Wolves,-2,Wolves,True,0
7,2025-08-02_Tunisia_Q9_Wolves,Tunisia,Wolves,Q9,-4,Q9,False,0
8,2025-08-03_Coastal_OUG_Q9,Coastal,Q9,OUG,4,Q9,False,1
9,2025-08-03_Firing Range_SPG_XROCK,Firing Range,SPG,XROCK,6,SPG,False,1


In [11]:
7/44*100

15.909090909090908